# Trotter Extrapolation

## James D. Watson, Jacob Watkins
## August 2023

### TODO:

- Generalize Trotter code to any order formula (currently first order), or at least 2nd order
- Calculate observable (Magnetization? Something local to one qubit?)

## Theory

From Equation (1) of [this text](https://arxiv.org/abs/1711.10980), we want to simulate the 1D Heisenberg model.

$$
H = \sum_{i=1}^n \sigma_i \cdot \sigma_{i+1} + h_j \sigma_j^z
$$

with $h_j \in [-h,h]$ randomly and uniformly sampled. Suppose we wish to simulate $e^{-i H T}$ on a quantum computer using product formulas. A natural choice is to split according to even-odd staggering of the interaction term, and then do the $Z$ rotations in their own grouping. If $n$ is odd, there will be one term left over. I don't think we should fret, probably just choose even $n$ for simplicity since we are only exploring our method. 

In [2]:
# Imports

import numpy as np
import scipy.linalg as sla
import numpy.linalg as nla

## Useful Pauli and qubit manipulations

In [3]:
# Computes Kronecker (tensor) product of a list of matrices
def kron_list(matrix_list):
    result = matrix_list[0]
    for i in range(1,len(matrix_list)):
        result = np.kron(result,matrix_list[i])
        
    return result

# Pauli matrices
I = np.array([[1.,0],[0,1.]], dtype='complex')
X = np.array([[0,1.],[1.,0]], dtype='complex')
Y = np.array([[0,-1.j],[1.j,0]], dtype='complex')
Z = np.array([[1.,0],[0,-1.]], dtype='complex')

# Converts string representation of paulis to list of matrices
def paulistring_to_list(paulistring): 
    matrix_list = list(paulistring)
    translate = {'I':I, 'X':X, 'Y':Y, 'Z':Z}
    for p in range(len(paulistring)):
        matrix_list[p] = translate[matrix_list[p]]
    return matrix_list

# Computes generalized pauli matrix, given a string in standard form
def pauli_matrix(paulistring):
    return kron_list(paulistring_to_list(paulistring))

# Computes generalized pauli matrix, specified by non-identity pieces. Nonidentities encoded as dictionary
# of the form {k:'P', ..., } where k is the integer location and P is a pauli
def sparse_pauli(nonidentities, nqubits):
    #starting string is all identity
    paulilist = []
    for i in range(0,nqubits):
        paulilist.append('I')
    
    #Change string to paulis specified by dictionary
    for key in nonidentities:
        paulilist[key] = nonidentities[key]
    paulistring = ''.join(paulilist)
    
    return pauli_matrix(paulistring) 

def sigma_dot_sigma(i,j,nqubits):
    return sparse_pauli({i:'X',j:'X'},nqubits) + sparse_pauli({i:'Y',j:'Y'},nqubits) + sparse_pauli({i:'Z',j:'Z'},nqubits)

# Qubit computational basis
zero = np.array([1.,0], dtype ='complex')
one = np.array([0,1.], dtype ='complex')

#Convert bitstring to list of qubit kets
def bitstring_to_list(bitstring):
    bitlist = list(bitstring)
    translate = {'0':zero, '1':one}
    for b in range(len(bitstring)):
        bitlist[b] = translate[bitlist[b]]
    return bitlist

def basis_ket(bitstring):
    return kron_list(bitstring_to_list(bitstring))

def sparse_bitstring(ones, nqubits):
    bitlist = []
    for i in range(0,nqubits):
        bitlist.append('0')
    
    #Flip certain bits to one, as specified by dictionary
    for qubit in ones:
        bitlist[qubit] = '1'
    bitstring = ''.join(bitlist)
    
    return basis_ket(bitstring)

## Chebyshev functions

`np.polynomial.chebyshev` is a class for chebyshev polynomials. Looks new and somewhat untested.

In [4]:
def cheb_nodes(n, width = 1, center = 0):
    nodes = np.zeros(n)
    for j in range(n):
        nodes[n-j-1] = center + (width/2)* np.cos((2*j+1)*np.pi/(2*n))
    return nodes

## Simulation functions

In [7]:
def heisenbergH(nqubits, hlist):
    H = sigma_dot_sigma(0,1,nqubits) + hlist[0]*sparse_pauli({0:'Z'}, nqubits)
    for j in range(1, nqubits):
        H += sigma_dot_sigma(j, (j+1)%nqubits, nqubits) + hlist[j]*sparse_pauli({j:'Z'}, nqubits)
    return H

def Uexact(H, T):
    return sla.expm(-1.j*H*T)

# TODO: generalize for higher order formulas (or at least 2).
# First term in Hlist is applied to state first
def Utrot_short(Hlist, t, order = 1):
    m = len(Hlist) # number of terms
    result = sla.expm(-1.j*Hlist[0]*t)
    for j in range(1,m):
        result = sla.expm(-1.j*Hlist[j]*t) @ result
    
    return result
    
# Code only works for first order
def Utrot_long(Hlist, T, steps = 1, order = 1):
    single_step = Utrot_short(Hlist, T/steps, order)
    return nla.matrix_power(single_step, steps)

In [16]:
# Parameters
nqubits = 6
h = 0
steps = 1000
T = 8

hlist = (np.random.rand(nqubits)*2 -1)*h

# Trotter pieces

Even = sigma_dot_sigma(0, 1, nqubits)
for j in range(2,nqubits,2):
    Even += sigma_dot_sigma(j, (j+1)%nqubits, nqubits)

Odd = sigma_dot_sigma(1, 2%nqubits, nqubits)
for j in range(3,nqubits,2):
    Odd += sigma_dot_sigma(j, (j+1)%nqubits, nqubits)
    
Potential = np.sum([hlist[j]*sparse_pauli({j:'Z'}, nqubits) for j in range(nqubits)], axis = 0)

Hlist = [Even, Odd, Potential]


A = Uexact(heisenbergH(nqubits, hlist), T)
B = Utrot_long(Hlist, T, steps)

In [17]:
# Unitaries don't line up. Hamiltonians seem to match.
sla.norm(A-B,2)

0.035174978485305756

In [99]:
# Hamiltonians line  up
la.norm(heisenbergH(nqubits, hlist) - np.sum(Hlist,axis=0))

0.0

In [100]:
2%10

2

In [81]:
for j in range(0,6,2):
    print(j)

0
2
4
